In [14]:
from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_votium_prices
from helpers.votium import cvx_mint_ratio, votium_round_emissions_after_fee, fxs_emissions_usd
from helpers.subgraph_endpoints import *
from helpers.formatter import pct_format, dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [16]:
cvx_price, crv_price, badger_price, fxs_price = get_votium_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(crv_price), "${:.2f}".format(cvx_price), "${:.2f}".format(fxs_price)]]
headers = ["BADGER price", "CRV price", "CVX price", "FXS price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,BADGER price,CRV price,CVX price,FXS price
0,$2.65,$0.70,$4.34,$4.48


In [17]:
# NOTE: modify with right proposal (latest & current)
proposal_id_last = "0x666833db066627cb5b9c67c8342df16d3763646f1c2c12865654e830a61ac857"
proposal_id_current = "0xd7db40d1ca142cb5ca24bce5d0f78f3b037fde6c7ebb3c3650a317e910278b1f"

cost_per_vote_after_fee = get_cost_per_vote_after_fee("bribes-overview-votium")

# check active voter in past proposal as ref
response_last_proposal = get_proposal_info(proposal_id_last)
response_current_proposal = get_proposal_info(proposal_id_current)

block_current_proposal = int(response_current_proposal["snapshot"])
cvx_voting_actively_last_round = response_last_proposal["scores_total"]
data = [[block_current_proposal,  f'{cvx_voting_actively_last_round:.0f}', dollar_format(cost_per_vote_after_fee)]]
headers = ["Proposal block height", "Past Round Active vlCVX votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlCVX votes,Cost per Vote Last Round
0,15935900,49714100,$0.0434


In [18]:
# emissions ecosystem
cvx = Contract('0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B')
cvx_ratio = cvx_mint_ratio(cvx, block_current_proposal)
biweekly_emissions= votium_round_emissions_after_fee(cvx_ratio, crv_price, cvx_price)
data = [[cvx_ratio, dollar_format(biweekly_emissions)]]
headers = ["Cvx Mint Crv Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions

,Cvx Mint Crv Ratio,Bi-weekly Emissions
0,0.042368,$5445789.7577


In [19]:
# vlCVX pct treasury breakdown
vlCVX = Contract("0x72a19342e8F1838460eBFCCEf09F6585e32db86E")
vlCVX_ts = vlCVX.totalSupply(block_identifier=block_current_proposal)/1e18
# check naked bvecvx belongings
bvecvx = Contract("0xfd05D3C7fe2924020620A8bE4961bBaA747e6305")
bvecvx_treasury_controlled = bvecvx.balanceOf("0xA9ed98B5Fb8428d68664f3C5027c62A10d45826b", block_identifier=block_current_proposal)/1e18
# check lock belongs in lp sett
cvx = Contract("0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B")
bvecvx_lp = Contract("0x04c90C198b2eFF55716079bc06d7CCc4aa4d7512")
bvecvx_cvx_sett = Contract("0x937B8E917d0F36eDEBBA8E459C5FB16F3b315551")
bvecvx_lp_treasury_controlled = bvecvx_cvx_sett.balanceOf("0xA9ed98B5Fb8428d68664f3C5027c62A10d45826b", block_identifier=block_current_proposal)/1e18
cvx_bal_in_lp = cvx.balanceOf(bvecvx_lp, block_identifier=block_current_proposal)/1e18
bvecvx_bal_in_lp = bvecvx.balanceOf(bvecvx_lp, block_identifier=block_current_proposal)/1e18
total_bal_in_lp = cvx_bal_in_lp + bvecvx_bal_in_lp
bve_cvx_pool_composition = bvecvx_bal_in_lp / total_bal_in_lp
treasury_votes = bvecvx_treasury_controlled + bvecvx_lp_treasury_controlled * bve_cvx_pool_composition

pct_controlled_cvx_by_badger_treasury = treasury_votes / vlCVX_ts

data = [[treasury_votes, pct_format(pct_controlled_cvx_by_badger_treasury)]]
headers = ["Treasury VP", "Badger Treasury CVX Controlled"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp


,Treasury VP,Badger Treasury CVX Controlled
0,95755.206358,0.212%


In [20]:
vecrv = Contract("0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2")
vecrv_ts = vecrv.totalSupplyAt(block_current_proposal) / 1e18
vlcrv_convex_controlled = vecrv.balanceOfAt("0x989AEb4d175e16225E39E87d0D97A3360524AD80", block_current_proposal)/1e18
pct_convex_vecrv_controlled = vlcrv_convex_controlled/vecrv_ts
pct_badger_vecrv_controlled = pct_controlled_cvx_by_badger_treasury*pct_convex_vecrv_controlled
data = [[pct_format(pct_convex_vecrv_controlled), pct_format(pct_badger_vecrv_controlled)]]
headers = ["Convex veCRV controlled", "Badger Treasury veCRV controlled"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Convex veCRV controlled,Badger Treasury veCRV controlled
0,53.072%,0.113%


In [21]:
# bribes allocation
bribe_for_badgerfraxbp = 6_000
bribe_usd_val = bribe_for_badgerfraxbp * badger_price
data = [[bribe_for_badgerfraxbp, bribe_usd_val]]
headers = ["Bribe allocated to badgerfraxbp pool (BADGER)", "Bribe Value (USD)"]
df_bribes_allocation = pd.DataFrame(data, columns=headers)
df_bribes_allocation

,Bribe allocated to badgerfraxbp pool (BADGER),Bribe Value (USD)
0,6000,15900.0


In [22]:
# badgerfraxbp prediction for vecrv relative weight given the bribe
vlCVX_votes_bought = bribe_usd_val / cost_per_vote_after_fee
pct_controlled_cvx_by_bribing_badgerfraxbp = vlCVX_votes_bought / vlCVX_ts
vecrv_expected_relative_weight_badgerfraxbp = pct_controlled_cvx_by_bribing_badgerfraxbp * pct_convex_vecrv_controlled
data = [[vlCVX_votes_bought, pct_format(vecrv_expected_relative_weight_badgerfraxbp)]]
headers = ["vlCVX badgerfraxbp pool", "veCRV relative weight"]
df_bribe_breakdown_badgereth = pd.DataFrame(data, columns=headers)
df_bribe_breakdown_badgereth


,vlCVX badgerfraxbp pool,veCRV relative weight
0,366359.447005,0.431%


In [23]:
# treasury expected earnings bi-weekly expectation post-bribe in badgerfraxbp pool
tvl_badgerfraxbp = get_tvl_curve_badgerfraxbp_pool()
badgerfraxbp_pool = Contract("0x09b2E090531228d1b8E3d948C73b990Cb6e60720")
pool_supply = badgerfraxbp_pool.totalSupply() / 1e18
private_vault_treasury = "0xa895B89D74a6BC23a284a0526e123ea776674cF5"
gauge_frax = Contract("0x5a92EF27f4baA7C766aee6d751f754EBdEBd9fae")
treasury_tcl_bal = gauge_frax.lockedLiquidityOf(private_vault_treasury)/1e18
vault_owned_tcl = tvl_badgerfraxbp * (treasury_tcl_bal/pool_supply)
pool_capture = vault_owned_tcl / tvl_badgerfraxbp
pool_earnings = biweekly_emissions * vecrv_expected_relative_weight_badgerfraxbp
pool_earnings_tcl_capture = pool_earnings * pool_capture
treasury_roi = pool_earnings_tcl_capture / bribe_usd_val
data = [[dollar_format(pool_earnings), dollar_format(tvl_badgerfraxbp), pct_format(pool_capture), dollar_format(poor_earnings_tcl_capture), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGERFRAXBP Pool TVL", "BADGERFRAXBP Pool Capture", "Treasury yield earned (USD) - biweekly", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool


,Pool yield (USD) - biweekly,BADGERFRAXBP Pool TVL,BADGERFRAXBP Pool Capture,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,$15849.0804,$2403165.9545,18.007%,$2853.9381,17.949%


In [24]:
# vefxs mirroring from convex, only known at proposal closing time
vefxs = Contract("0xc8418aF6358FFddA74e09Ca9CC3Fe03Ca6aDC5b0")
vefxs_ts = vefxs.totalFXSSupply() / 1e18
vefxs_convex_bal = (
    vefxs.locked("0x59CFCD384746ec3035299D90782Be065e466800B")["amount"] / 1e18
)
vefxs_convex_controlled = vefxs_convex_bal / vefxs_ts
vefxs_mirror_weight = get_vefxs_mirror_weight(response_current_proposal)
treasury_fxs_gauge_capture = treasury_tcl_bal / (
    gauge_frax.totalLiquidityLocked() / 1e18
)
gauge_earnings = fxs_emissions_usd(
    fxs_price, vefxs_convex_controlled * vefxs_mirror_weight, treasury_fxs_gauge_capture
)
# NOTE: use pool earnings we're capturing not total earnings!
total_earnings = poor_earnings_tcl_capture + gauge_earnings
treasury_roi = total_earnings / bribe_usd_val
data = [
    [
        pct_format(vefxs_convex_controlled),
        pct_format(treasury_fxs_gauge_capture),
        pct_format(vefxs_mirror_weight),
        vefxs_convex_bal * vefxs_mirror_weight,
        dollar_format(total_earnings),
        pct_format(treasury_roi)
    ]
]
headers = [
    "veFXS Convex controlled",
    "Treasury FXS gauge capture",
    "veFXS convex mirroring weigth",
    "veFXS convex mirroring VP",
    "Treasury yield earned (USD) - biweekly",
    "Treasury ROI - biweekly"
]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,veFXS Convex controlled,Treasury FXS gauge capture,veFXS convex mirroring weigth,veFXS convex mirroring VP,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,16.439%,18.117%,1.670%,110852.574792,$3177.6056,19.985%
